We are Turning List elements into One Hot Vectors

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
pd.__version__


'2.2.2'

In [2]:
games_df = pd.read_csv('../Data/games.csv')
# games_df.info()


In [3]:
new_df = games_df['Categories'].fillna('').str.removeprefix('[').str.removesuffix(']').str.split(',').to_frame()
new_df.head(10)
# new_df.dtypes


,Categories
0,"[Single-player, Multi-player, Steam Achievemen..."
1,"[Single-player, Steam Achievements, Full contr..."
2,[Single-player]
3,"[Single-player, Full controller support]"
4,"[Single-player, Steam Achievements]"
5,"[Single-player, Multi-player, MMO, PvP, Online..."
6,"[Single-player, Steam Achievements, Steam Cloud]"
7,"[Single-player, Steam Cloud]"
8,"[Single-player, Steam Achievements, Full contr..."
9,"[Single-player, Steam Achievements, Steam Trad..."


In [4]:
binarize_col_names = ['Categories', 'Genres', 'Tags', 'Supported languages', 'Full audio languages']
split_col_first = [True, True, True, False, False]

output_df = pd.DataFrame()

for idx, column in enumerate(binarize_col_names):
	# print(column)
	mlb = MultiLabelBinarizer(sparse_output=True)
	if split_col_first[idx]:
		new_df = games_df[column].fillna('').str.removeprefix('[').str.removesuffix(']').str.split(',').to_frame()

	else:
		new_df = games_df[column].fillna('').str.rstrip().str.replace("'", "").str.removesuffix("' ").str.removeprefix('[]').str.removeprefix("[").str.removesuffix("]").str.split(",").to_frame()

	# new_df = new_df[column].to_frame()

		
	new_df = pd.DataFrame.sparse.from_spmatrix(
						mlb.fit_transform(new_df.pop(column)),
						columns=mlb.classes_)
		
	new_df = new_df.add_prefix(column)
	# print(new_df.columns)
	output_df = output_df.join(new_df, how='outer')
	# print(output_df.head())
	
	# output_df.head(3)
	
output_df.head(10)
# output_df.memory_usage


,Categories,CategoriesCaptions available,CategoriesCo-op,CategoriesCommentary available,CategoriesCross-Platform Multiplayer,CategoriesFull controller support,CategoriesIn-App Purchases,CategoriesIncludes Source SDK,CategoriesIncludes level editor,CategoriesLAN Co-op,...,Full audio languagesSimplified Chinese,Full audio languagesSlovak,Full audio languagesSpanish - Latin America,Full audio languagesSpanish - Spain,Full audio languagesSwedish,Full audio languagesThai,Full audio languagesTraditional Chinese,Full audio languagesTurkish,Full audio languagesUkrainian,Full audio languagesVietnamese
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# remove blank columns
output_df = output_df.drop(binarize_col_names, axis=1)


In [6]:
output_df.memory_usage()


Index                                        128
CategoriesCaptions available                8936
CategoriesCo-op                            53336
CategoriesCommentary available              1808
CategoriesCross-Platform Multiplayer       16152
                                           ...  
Full audio languagesThai                      32
Full audio languagesTraditional Chinese      528
Full audio languagesTurkish                  232
Full audio languagesUkrainian                 80
Full audio languagesVietnamese                24
Length: 834, dtype: int64

In [7]:
output_df.to_csv('../Data/OneHotVectors.csv')
